In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
from scipy import optimize

In [2]:
np.random.seed(seed=42)

In [5]:
age_and_sex = pd.read_csv('age_and_sex.csv').drop(['Unnamed: 0'], axis = 1)

In [7]:
age_and_sex.values

array([[ 2.1,  1. ],
       [25.6,  0. ],
       [23.6,  0. ],
       ...,
       [ 2. ,  0. ],
       [41.5,  1. ],
       [21. ,  0. ]])

In [ ]:
class Tucker_Model():
    def __init__(self,
                 num_ppl_iso, 
                 mean_iso, 
                 prop_iso, 
                 num_ppl_tents, 
                 max_tent,
                 prob_house,
                 prob_meet,
                 init_trans,
                 prob_symp,
                 recovery,
                 perm_asymp,
                 int_eth,
                 small_rad,
                 large_rad,
                 rad_int,
                 filename='age_and_sex.csv'):
        
        self.num_ppl_iso = num_ppl_iso # Number of people in isoboxes. (Nb)
        self.mean_iso = mean_iso # Isoboxes mean occupancy (people). (mub)
        self.num_iso = num_ppl_iso/mean_iso # Number of isoboxes. (hb)
        self.prop_iso = prop_iso # Proportion of area covered by isoboxes. (iba)
        self.num_ppl_tents = num_ppl_tents # Number of people in tents. (Nt)
        self.max_tent = max_tent # Tents occupancy of (people). (mut)
        self.num_tents = num_ppl_tents/max_tent # Number of tents. (ht)
        self.total_pop = num_ppl_tents + num_ppl_iso # Total population. (N)
        self.prob_house = prob_house # Probability of infecting each person in your household per day. (twh)
        self.prob_meet = prob_meet # Probability of infecting each person you meet per meeting (Fang et al.) (aip)
        self.init_trans = init_trans # Initial transmission reduction (relative to assumed per contact transmission rate, outside household only). (tr)
        self.prob_symp = prob_symp # Probability of spotting symptoms, per person per day. (siprob)
        self.recovery = recovery # Days in quarantine after no virus shedding (i.e., recovery). (clearday)
        self.perm_asymp = perm_asymp # Permanently asymptomatic cases (Mizumoto et al 2020 Eurosurveillance). (pac)
        self.int_eth = int_eth # Realtive strength of interaction between different ethnicities. (ss)
        self.small_rad = small_rad # Smaller movement radius. Range around their household during lockdown or females and individuals age < 10. (lr1)
        self.large_rad = large_rad # Larger movement radius. ie. Pople who violate lockdown enforcement or males over age 10. (lr2)
        self.rad_int = rad_int # Scale interactions - two people with completely overlapping rages with this radius interact once per day (lrtol)
        self.data = pd.read_csv(filename).drop(['Unnamed: 0'], axis = 1).values
        
    def create_matrix(self):
        """
        Creates N x 9 matrix, with the following column definitions:
        
        Columns:
        0. Home number
        1. Disease state: 0 = susceptible, 1 = exposed, 2 = presymptomatic, 3 = symptomatic, 4 = mild, 5 = severe, 6 = recovered. Similar states in quarentine are the same plus seven. In other words, this is a categorical variable with values between 0 and 13 inclusive.
        2. Days to symptoms for this person
        3. Days passed in current state
        4. Whether this person will be asymptomatic
        5. Age
        6. Male: = 1 if male.
        7. Chronic: = 1 if chronic disease?
        8. Wanderer (Uses the larger radius)
        
        This is the primary object that is operated on within the class. It will model the epidemic within the community
        by adjusting the parameters in the weekend.
        """
        
        column_1 = np.subtract(np.concatenate((np.ceil(self.num_iso*np.random.uniform(0,1,self.num_people_iso)), 
                                               self.num_iso+np.ceil(self.num_tents*np.random.uniform(0,1,Nt)))),1).astype(int)
        column_2 = 